In [2]:
"""
Refactored FastAPI Application for Mahasiswa Management
This application provides API endpoints to manage student (mahasiswa) data
"""

# Install library (jalankan ini untuk menginstall dependencies)
!pip install fastapi uvicorn nest-asyncio pyngrok

# Import libraries
from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn
from typing import List, Dict, Any, Optional

# Models
class Mahasiswa(BaseModel):
    """Model representing a student with name and student ID"""
    nama: str
    nim: str

class ResponseMessage(BaseModel):
    """Standard response model for API responses"""
    pesan: str
    data: Optional[Dict[str, Any]] = None

# Initialize data
class MahasiswaRepository:
    """Repository to manage mahasiswa data"""
    def __init__(self):
        self.mahasiswa_list = [
            {"nama": "Afif Ramadhani", "nim": "2211104066"},
            {"nama": "LeBron James", "nim": "1302000001"},
            {"nama": "Stephen Curry", "nim": "1302000002"},
            {"nama": "Kobe Bryant", "nim": "1302000003"},
        ]

    def get_all(self) -> List[Dict[str, str]]:
        """Get all mahasiswa records"""
        return self.mahasiswa_list

    def get_by_index(self, index: int) -> Dict[str, str]:
        """Get a mahasiswa by index"""
        if 0 <= index < len(self.mahasiswa_list):
            return self.mahasiswa_list[index]
        return None

    def add(self, mahasiswa: Mahasiswa) -> None:
        """Add a new mahasiswa"""
        self.mahasiswa_list.append(mahasiswa.dict())

    def delete(self, index: int) -> Dict[str, str]:
        """Delete a mahasiswa by index"""
        if 0 <= index < len(self.mahasiswa_list):
            return self.mahasiswa_list.pop(index)
        return None

# Create the FastAPI application
def create_app() -> FastAPI:
    """Create and configure the FastAPI application"""
    app = FastAPI(
        title="Mahasiswa API",
        description="API for managing student data",
        version="1.0.0"
    )

    # Create repository instance
    repo = MahasiswaRepository()

    # Register routes
    @app.get("/api/mahasiswa", response_model=List[Dict[str, str]])
    def get_semua_mahasiswa():
        """Get all mahasiswa data"""
        return repo.get_all()

    @app.get("/api/mahasiswa/{index}")
    def get_mahasiswa(index: int):
        """Get mahasiswa by index"""
        mahasiswa = repo.get_by_index(index)
        if mahasiswa:
            return mahasiswa
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Index tidak ditemukan"
        )

    @app.post("/api/mahasiswa", response_model=ResponseMessage)
    def tambah_mahasiswa(mahasiswa: Mahasiswa):
        """Add a new mahasiswa"""
        repo.add(mahasiswa)
        return ResponseMessage(pesan="Mahasiswa berhasil ditambahkan")

    @app.delete("/api/mahasiswa/{index}", response_model=ResponseMessage)
    def hapus_mahasiswa(index: int):
        """Delete mahasiswa by index"""
        deleted = repo.delete(index)
        if deleted:
            return ResponseMessage(
                pesan="Mahasiswa berhasil dihapus",
                data=deleted
            )
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Index tidak ditemukan"
        )

    return app

def start_server():
    """Configure and start the server with ngrok tunnel"""
    # Configure ngrok
    conf.get_default().auth_token = "2wFymwB49ivdxPUxcGRsyEZF9OM_88dueVUsHUg3BtJjrGAL"

    # Apply nest_asyncio to allow asyncio to work in Jupyter
    nest_asyncio.apply()

    # Create ngrok tunnel
    public_url = ngrok.connect(8000)
    print("🚀 Swagger UI:", public_url.public_url + "/docs")
    print("🚀 API URL:", public_url.public_url)

    # Run the server
    app = create_app()
    uvicorn.run(app, port=8000)

# Run the application
if __name__ == "__main__":
    start_server()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00


INFO:     Started server process [701]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🚀 Swagger UI: https://2b00-35-236-252-114.ngrok-free.app/docs
🚀 API URL: https://2b00-35-236-252-114.ngrok-free.app
INFO:     103.47.133.146:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.47.133.146:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.47.133.146:0 - "GET /api/mahasiswa HTTP/1.1" 200 OK
INFO:     103.47.133.146:0 - "POST /api/mahasiswa HTTP/1.1" 200 OK


<ipython-input-2-420db5805d75>:51: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  self.mahasiswa_list.append(mahasiswa.dict())


INFO:     103.47.133.146:0 - "GET /api/mahasiswa HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [701]
